In [1]:
# Important : to make initial dataframe need to run RFMDash.ipynb and create TrainRFMRFMBL

%run "TrainInJupOnlineShopping.ipynb"


C:\Users\BiG\AppData\Local\Temp\ipykernel_6784\2681434285.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DimDateS["Miladi3"] = DimDateS["Miladi3"].astype(str)
C:\Users\BiG\AppData\Local\Temp\ipykernel_6784\2681434285.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DimDateS.rename(columns={"Miladi3": "date_CHR"}, inplace=True)


Index(['_id', 'user_id', 'date', 'client_type', 'status', 'payment_status',
       'type', 'module', 'product_name', 'initial_total', 'product_price',
       'converted_value', 'qty', 'module_unit_price', 'Amount_in_dollars',
       'first_purchased', 'date_CHR', 'Jalali_1', 'Shamsi_Date_Num'],
      dtype='object')
(503726, 19)


In [3]:
import pandas as pd
data = TrainRFMRFMBL[['user_id', 'R', 'F', 'M', 'RNormScore', 'FNormScore', 'MNormScore']]
df = pd.DataFrame(data)

In [4]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

app = Dash()


app.layout = html.Div([
    html.Div(
        children="مانیتورینگ رفتار پویای مشتریان", #/# Add Font next
        style={
            'backgroundColor': 'lightblue',
            'padding': '10px',
            'fontSize': '20px',
            'textAlign': 'center',
            'fontWeight': 'bold'
        }
    ),
    # html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df['user_id'].unique(), id = 'dropdown_selection'),
    dcc.Graph(id='barchart_usersRFM')
])

@callback(
    Output('barchart_usersRFM', 'figure'),
    Input('dropdown_selection', 'value')
)

def update_bar_chart(value):
    dff = df[df['user_id'] == value]
    fig = px.bar(
                x=['RNormScore', 'FNormScore', 'MNormScore'],
                y=[dff['RNormScore'].iloc[0], dff['FNormScore'].iloc[0], dff['MNormScore'].iloc[0]],
                labels={'x': 'Metric', 'y': 'Normalized Score'},
                title="Normalized Scores",
                color_discrete_sequence=['lightgreen']
            )
    return fig


if __name__ == '__main__':
    app.run(debug=True)


In [4]:
# import dash
# from dash import dcc, html
# from dash.dependencies import Input, Output
# import pandas as pd
# import plotly.express as px
#
# # فرض می‌کنیم TrainRFMRFMBL قبلاً تعریف شده است.
# data = TrainRFMRFMBL[['user_id', 'R', 'F', 'M', 'RNormScore', 'FNormScore', 'MNormScore']]
#
# # تبدیل داده به DataFrame
# df = pd.DataFrame(data)
#
# # ساخت اپلیکیشن Dash
# app = dash.Dash(__name__, suppress_callback_exceptions=True)  # اضافه کردن این گزینه
#
# app.layout = html.Div([
#     # نوار بالای داشبورد
#     html.Div(
#         children="مانیتورینگ رفتار مشتریان",
#         style={
#             'backgroundColor': 'lightblue',
#             'padding': '10px',
#             'fontSize': '20px',
#             'textAlign': 'center',
#             'fontWeight': 'bold'
#         }
#     ),
#
#     # لای اوت اصلی با دو بخش: تب‌ها و محتویات
#     html.Div([
#         # نوار تب‌ها
#         dcc.Tabs(id='tabs', value='tab1', children=[
#             dcc.Tab(label='tab1', value='tab1'),
#             dcc.Tab(label='tab2', value='tab2'),
#             dcc.Tab(label='tab3', value='tab3'),
#             dcc.Tab(label='tab4', value='tab4'),
#         ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),
#
#         # محتویات تب‌ها
#         html.Div(id='tab-content', style={'width': '75%', 'display': 'inline-block'})
#     ], style={'display': 'flex'})
# ])
#
# # محتویات هر تب
# @app.callback(
#     Output('tab-content', 'children'),
#     Input('tabs', 'value')
# )
# def render_tab(tab):
#     if tab == 'tab1':
#         dropdown = dcc.Dropdown(
#             id='user-id-dropdown',
#             options=[{'label': i, 'value': i} for i in df['user_id'].unique()],
#             value=df['user_id'].iloc[0],  # مقدار اولیه
#         )
#
#         # جدول برای R, F, M
#         table = html.Table([
#             html.Tr([html.Th("Feature"), html.Th("Value")]),
#             html.Tr([html.Td("R"), html.Td(df['R'].iloc[0])]),
#             html.Tr([html.Td("F"), html.Td(df['F'].iloc[0])]),
#             html.Tr([html.Td("M"), html.Td(df['M'].iloc[0])]),
#         ])
#
#         # بارچارت برای RNormScore, FNormScore, MNormScore
#         bar_chart = dcc.Graph(
#             figure=px.bar(
#                 x=['RNormScore', 'FNormScore', 'MNormScore'],
#                 y=[df['RNormScore'].iloc[0], df['FNormScore'].iloc[0], df['MNormScore'].iloc[0]],
#                 labels={'x': 'Metric', 'y': 'Normalized Score'},
#                 title="Normalized Scores",
#                 color_discrete_sequence=['lightgreen']
#             )
#         )
#
#         return html.Div([
#             # پنل کرکره ای
#             html.Div([
#                 html.Label('Select User ID'),
#                 dropdown
#             ], style={'width': '30%', 'padding': '10px', 'display': 'inline-block'}),
#
#             # پنل‌های عمودی
#             html.Div([
#                 # پنل چپ (جدول)
#                 html.Div([table], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
#                 # پنل راست (بارچارت)
#                 html.Div([bar_chart], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'})
#             ], style={'display': 'flex', 'justifyContent': 'space-between'})
#         ])
#     elif tab == 'tab2':
#         return html.Div("Content for tab2")
#     elif tab == 'tab3':
#         return html.Div("Content for tab3")
#     elif tab == 'tab4':
#         return html.Div("Content for tab4")
#
# if __name__ == '__main__':
#     app.run_server(debug=True)
